In [1]:
from paddlenlp.transformers import ErnieModel,ErnieTokenizer
from model import ErnieGRUCRF
from data import load_dataset,load_dict_single

In [2]:
train_ds, dev_ds = load_dataset(datafiles=('./data/train.0', './data/dev.0'))

# 把类别名转为id
label_vocab = load_dict_single('./data/tag.txt')

# 查看训练集和测试集的大小
print("训练集大小:", len(train_ds))
print("测试集集大小:", len(dev_ds))
print(train_ds[0])
print(dev_ds[0])
print(label_vocab)  # 57个分类

训练集大小: 67624
测试集集大小: 27715
(['浙', '江', '省', '杭', '州', '市'], ['B-prov', 'I-prov', 'E-prov', 'B-city', 'I-city', 'E-city'])
(['浙', '江', '省', '嗯', ','], ['B-prov', 'I-prov', 'E-prov', 'O', 'O'])
{'O': 0, 'B-prov': 1, 'I-prov': 2, 'E-prov': 3, 'B-city': 4, 'I-city': 5, 'E-city': 6, 'B-district': 7, 'I-district': 8, 'E-district': 9, 'B-town': 10, 'I-town': 11, 'E-town': 12, 'B-community': 13, 'I-community': 14, 'E-community': 15, 'B-poi': 16, 'E-poi': 17, '': 18, 'B-road': 19, 'E-road': 20, 'B-roadno': 21, 'I-roadno': 22, 'E-roadno': 23, 'I-poi': 24, 'B-assist': 25, 'E-assist': 26, 'B-distance': 27, 'E-distance': 28, 'I-road': 29, 'B-intersection': 30, 'I-intersection': 31, 'E-intersection': 32, 'S-assist': 33, 'B-subpoi': 34, 'I-subpoi': 35, 'E-subpoi': 36, 'B-devzone': 37, 'I-devzone': 38, 'E-devzone': 39, 'B-houseno': 40, 'I-houseno': 41, 'E-houseno': 42, 'I-distance': 43, 'B-road2': 44, 'I-road2': 45, 'E-road2': 46, 'I-assist': 47, 'B-cellno': 48, 'I-cellno': 49, 'E-cellno': 50, 'B-floo

In [3]:
pretrained_model = ErnieModel.from_pretrained(r"ernie-3.0-medium-zh")
tokenizer = ErnieTokenizer.from_pretrained("ernie-3.0-medium-zh")

[2024-03-05 18:27:43,144] [    INFO] - Already cached C:\Users\huangkai\.paddlenlp\models\ernie-3.0-medium-zh\model_state.pdparams
[2024-03-05 18:27:43,145] [    INFO] - Loading weights file model_state.pdparams from cache at C:\Users\huangkai\.paddlenlp\models\ernie-3.0-medium-zh\model_state.pdparams
[2024-03-05 18:27:43,740] [    INFO] - Loaded weights file from disk, setting weights to model.
[2024-03-05 18:27:59,985] [ WARNING] - Some weights of the model checkpoint at ernie-3.0-medium-zh were not used when initializing ErnieModel: ['ernie.encoder.layers.6.norm1.bias', 'ernie.encoder.layers.6.linear1.bias', 'ernie.encoder.layers.6.norm2.weight', 'ernie.encoder.layers.6.self_attn.v_proj.weight', 'ernie.encoder.layers.6.self_attn.v_proj.bias', 'ernie.encoder.layers.6.self_attn.out_proj.weight', 'ernie.encoder.layers.6.self_attn.k_proj.bias', 'ernie.encoder.layers.6.norm2.bias', 'ernie.encoder.layers.6.self_attn.out_proj.bias', 'ernie.encoder.layers.6.linear1.weight', 'ernie.encoder.l

In [4]:
model = ErnieGRUCRF(pretrained_model, 300, len(label_vocab), 100)
for params in model.gru.parameters():
    print(params)

Parameter containing:
Tensor(shape=[900, 768], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[ 0.01313203,  0.04926811,  0.00457385, ...,  0.00858852,
         -0.03788830,  0.00592746],
        [-0.00756706,  0.03596122, -0.05447865, ...,  0.03204236,
          0.00426644,  0.00759441],
        [-0.02786654,  0.04381340, -0.02210343, ...,  0.03279728,
         -0.03047972, -0.05408769],
        ...,
        [-0.04864771, -0.00919693,  0.05261052, ..., -0.04903826,
         -0.04365531,  0.01323290],
        [-0.04010753,  0.01352503, -0.01392372, ..., -0.02435323,
         -0.00707729,  0.04662792],
        [ 0.01341401, -0.01109541,  0.03490317, ...,  0.03130861,
         -0.02430127,  0.01336824]])
Parameter containing:
Tensor(shape=[900, 300], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[-0.04921215,  0.02503379, -0.04477245, ..., -0.04151532,
          0.05267966, -0.04388599],
        [-0.05123978,  0.01610538, -0.00299846, ..., -0.01225020,
    

In [5]:
import paddle
state_dict = paddle.load("ner_model/model_42280.pdparams")
model.set_dict(state_dict)

C:\Users\huangkai\AppData\Local\Programs\Python\Python39\lib\site-packages\paddle\nn\layer\layers.py:2084: UserWarning: Skip loading for fc.weight. fc.weight receives a shape [600, 58], but the expected shape is [600, 89].
  warnings.warn(f"Skip loading for {key}. " + str(err))
C:\Users\huangkai\AppData\Local\Programs\Python\Python39\lib\site-packages\paddle\nn\layer\layers.py:2084: UserWarning: Skip loading for fc.bias. fc.bias receives a shape [58], but the expected shape is [89].
  warnings.warn(f"Skip loading for {key}. " + str(err))
C:\Users\huangkai\AppData\Local\Programs\Python\Python39\lib\site-packages\paddle\nn\layer\layers.py:2084: UserWarning: Skip loading for crf.transitions. crf.transitions receives a shape [58, 58], but the expected shape is [89, 89].
  warnings.warn(f"Skip loading for {key}. " + str(err))
C:\Users\huangkai\AppData\Local\Programs\Python\Python39\lib\site-packages\paddle\nn\layer\layers.py:2084: UserWarning: Skip loading for crf_loss.crf.transitions. crf_

(['fc.weight',
  'fc.bias',
  'crf.transitions',
  'crf_loss.crf.transitions',
  'viterbi_decoder.transitions'],
 ['fc.weight',
  'fc.bias',
  'crf.transitions',
  'crf_loss.crf.transitions',
  'viterbi_decoder.transitions'])

In [6]:
for params in model.gru.parameters():
    print(params)

Parameter containing:
Tensor(shape=[900, 768], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[ 0.00431278,  0.00782988, -0.01447922, ..., -0.03163040,
          0.02313145,  0.03078911],
        [-0.04491691, -0.00364067, -0.00983983, ..., -0.01720943,
          0.07097724, -0.00365569],
        [-0.06247249,  0.01491211, -0.04569069, ..., -0.02207063,
         -0.06114850,  0.04144102],
        ...,
        [-0.01824863,  0.01792793, -0.05014712, ...,  0.00244747,
          0.03399225, -0.03012270],
        [ 0.04483406, -0.07919921,  0.01683323, ..., -0.05874594,
         -0.06198655,  0.01496401],
        [-0.05167799,  0.03493969,  0.01011887, ...,  0.05697599,
          0.01123167, -0.04701636]])
Parameter containing:
Tensor(shape=[900, 300], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[-0.01338924, -0.04375196,  0.00523560, ...,  0.04217119,
         -0.00599962,  0.04805801],
        [-0.04411605, -0.04818212, -0.01897270, ...,  0.01614543,
    

In [7]:
import paddle
import paddle.nn as nn
from paddlenlp.data import Pad, Tuple
def convert_example(example, tokenizer, max_seq_length=512):
    query = example

    query_encoded_inputs = tokenizer(text=query, max_seq_len=max_seq_length)
    query_input_ids = query_encoded_inputs["input_ids"]
    query_token_type_ids = query_encoded_inputs["token_type_ids"]

    # title_encoded_inputs = tokenizer(text=title, max_seq_len=max_seq_length)
    # title_input_ids = title_encoded_inputs["input_ids"]
    # title_token_type_ids = title_encoded_inputs["token_type_ids"]

    return query_input_ids, query_token_type_ids  # , title_input_ids, title_token_type_ids

In [10]:
ernie=model.Ernie
ernie.eval()

In [12]:
data1 = "殊不知你爹我一个不小心用力过猛一巴掌将你婊子妈的狗头拍出脑震荡成了真正的白癡"
query_input_ids, query_token_type_ids = convert_example(data1, tokenizer, max_seq_length=128)
query_input_ids = Pad(axis=0, pad_val=tokenizer.pad_token_id)([query_input_ids])
query_token_type_ids = Pad(axis=0, pad_val=tokenizer.pad_token_type_id)([query_token_type_ids])
query_input_ids = paddle.to_tensor(query_input_ids)
query_token_type_ids = paddle.to_tensor(query_token_type_ids)
result1 = ernie(query_input_ids, query_token_type_ids)

In [14]:
result1[1]

Tensor(shape=[1, 768], dtype=float32, place=Place(cpu), stop_gradient=True,
       [[-0.34770352, -0.32182032,  0.49859843, -0.26620653, -0.69830751,
         -0.52632791,  0.23800817,  0.22080097,  0.02169212,  0.47703984,
         -0.26219422,  0.37433460,  0.49425903,  0.22586782,  0.43094891,
          0.06630542,  0.42533305, -0.14868617, -0.07678396,  0.13735738,
          0.07371775,  0.32851204,  0.62721199,  0.23722085, -0.17046942,
         -0.12202432, -0.35211173,  0.11635464, -0.32354790, -0.16836995,
         -0.21451342,  0.38062981,  0.31306416,  0.14954662, -0.04986373,
         -0.48808187, -0.04935552, -0.21617655, -0.03744492, -0.35693356,
         -0.09559148, -0.05473252,  0.03617846,  0.27472371,  0.39588618,
         -0.26838797,  0.16297279,  0.45378485,  0.18148063, -0.30583885,
          0.49566910,  0.11761373, -0.37454683, -0.43818048, -0.17410117,
          0.53982705, -0.43336657, -0.46590531, -0.15718058,  0.15658386,
         -0.05438850, -0.58335817, -

In [15]:
class GRUCRF(nn.Layer):
    def __init__(self,):
        super().__init__()
        self.gru = model.gru
        self.fc = model.fc
        self.crf = model.crf
        self.viterbi_decoder = model.viterbi_decoder

    def forward(self,encoder_output,lengths=None,labels=None):
        gru_output, _ = self.gru(encoder_output)
        emission = self.fc(gru_output)
        _,prediction = self.viterbi_decoder(emission, lengths)
        return prediction

In [ ]:
final_model=GRUCRF.eval()

